# Compile Full qPCR Repeats

31 May 2016

In [91]:
library(lubridate)
library(dplyr) 
library(data.table) 
library(ggplot2) 
library(readr) 
library(tidyr) 
library(magrittr)
library(openxlsx)

In [92]:
library(iplantAcquiR)
handle <- iplant_handle(username = Sys.getenv("USER"),
                        password = Sys.getenv("IPLANT_PASSWORD"))
handle <- iplant_authenticate(handle)

Submitting with 'NULL'


In [93]:
repeat_filename1 <- iplant_acquire(handle = handle,
                           url = "https://iplant.plantandfood.co.nz/project/I130806/Research/GEP_repeats%20_qPCR_results.xlsx")
repeat_filename2 <- iplant_acquire(handle = handle,
                           url = "https://iplant.plantandfood.co.nz/project/I130806/Research/GEP_final_repeat_qPCR_results_omit_GEP3+4.xlsx")


### Read Main Repeats Workbook

In [94]:
repeat_filename1

[1] "GEP_repeats__qPCR_results.xlsx"

In [95]:
(sheetnames1 <- getSheetNames(repeat_filename1))

[1] "RNA plate order"                 "GEP_repeat_cDNA_layout"         
[3] "96 to 384 well for repeat plate" "GEP_repeats_VRN1_results"       
[5] "GEP_repeats_VRN2_results"        "GEP_repeats_VRN3_results"       
[7] "GEP_repeats_EGF1a_results"       "GEP_repeats_Ta54227_results"

### Read Well Config

 Need **two** sheets here
 
 1. 96 to 384 well mapping
 2. mapping from original sample plates to new repeat plate
 

In [96]:
trans_dt <- data.table(read.xlsx(repeat_filename1,'96 to 384 well for repeat plate' ))
setnames(trans_dt,names(trans_dt),gsub('(\\.)+','_',names(trans_dt)))
head(trans_dt)

,sample_well,qPCR_well
1,A1,A1
2,A1,B1
3,A1,A2
4,A6,B12
5,A6,A11
6,A6,B11


In [101]:
trans_dt[sample_well=='IRC']

,sample_well,qPCR_well
1,IRC,B24
2,IRC,D24
3,IRC,F24


In [133]:
cDNA_dt <- data.table(read.xlsx(repeat_filename1,'GEP_repeat_cDNA_layout' ))
setnames(cDNA_dt,names(cDNA_dt),gsub('(\\.)+','_',names(cDNA_dt)))
setnames(cDNA_dt,'Sample','Name')
head(cDNA_dt)

,repeat_plate_template_well,Name,ex_sample_plate,ex_sample_plate_well
1,A1,AMK258,GEP1,12G
2,B1,SCN13,GEP2,2B
3,C1,BW92,GEP2,2D
4,D1,CRWT196,GEP2,2H
5,E1,AMK320,GEP2,3A
6,F1,CRWT240,GEP2,3B


In [134]:
cDNA_dt[,unique(Name)]

[1] "AMK258"   "SCN13"    "BW92"     "CRWT196"  "AMK320"   "CRWT240" 
 [7] "SCN265"   "SCN142"   "BW47"     "CRWT232"  "BW219"    "AMK126"  
[13] "CRWT42"   "CRWT151"  "AMK69"    "SCN131"   "AMK275"   "SCN310"  
[19] "CRWT271"  "SCN71"    "AMK204"   "AMK20"    "AMK293"   "CRWT318" 
[25] "AMK330"   "AMK76"    "CRWT274"  "BW155"    "AMK156"   "CRWT345" 
[31] "CRWT293"  "AMK180"   "BW237"    "BW265"    "BW244"    "OT30"    
[37] "SCN146"   "OT21"     "SCN136"   "OT37"     "OT8"      "CRWT316" 
[43] "CRWT336"  "AMK214"   "AMK196"   "BS73"     "AMK212"   "AMK185"  
[49] "BW264"    "AMK1154"  "AMK188"   "AMK199"   "AMK198"   "CRWT300" 
[55] "BW286"    "BW217"    "AMK176"   "AMK184"   "OT1026"   "SCN1115" 
[61] "BW239"    "CRWT307"  "OT220"    "BW192"    "BW287"    "CRWT65"  
[67] "BW256"    "CRWT38"   "CRWT145"  "BW214"    "BW279"    "AMK325"  
[73] "BS47"     "AMK207"   "CRWT1244" "AMK1156"  "BW96"     "BW33"    
[79] "SCN290"   "AMK114"   "AMK22"    "CRWT216"  "BW277"    "OT36"    
[85] "BW218"    "CRWT310"  "CRWT1255" "AMK157"   "CRWT1258" "IRC"

### Read the Data Sheets

In [104]:
(dataSheets <- sheetnames1[grepl("_results$", sheetnames1)])

[1] "GEP_repeats_VRN1_results"    "GEP_repeats_VRN2_results"   
[3] "GEP_repeats_VRN3_results"    "GEP_repeats_EGF1a_results"  
[5] "GEP_repeats_Ta54227_results"

In [105]:
# read sheet X frpom xlsx at filepath and return a data.table
read_qPCR <- function(filepath,X) {
  require(openxlsx)
  require(data.table)
  require(dplyr)
  wb_path <- filepath
  my_sheet_data <- strsplit(X,split='_')[[1]]
  my_plate <- my_sheet_data[1]
  my_gene <- my_sheet_data[3]
  my_df <- data.table(read.xlsx(wb_path,X,startRow=2))
  my_df$gene <- my_gene
  my_df$plate <- my_plate
  setnames(my_df,c('Pos'),c('qPCR_well'))
  my_df %<>% 
    inner_join(trans_dt,by='qPCR_well') %>%
  arrange(sample_well) %>% data.table
  my_df<- suppressWarnings(my_df[,grep("^Tm", colnames(my_df)):= NULL,with=FALSE])
  return(as.data.table(my_df))
}

In [106]:
read_qPCR(repeat_filename1,'GEP_repeats_Ta54227_results') %>% head

,qPCR_well,Name,Cp,gene,plate,sample_well
1,A1,AMK258,24.83,Ta54227,GEP,A1
2,A2,AMK258,24.83,Ta54227,GEP,A1
3,B1,AMK258,24.9,Ta54227,GEP,A1
4,A19,CRWT145,24.5,Ta54227,GEP,A10
5,A20,CRWT145,24.59,Ta54227,GEP,A10
6,B19,CRWT145,NA,Ta54227,GEP,A10


In [107]:
repeat_result_dt <- rbindlist(lapply(dataSheets,function(S)read_qPCR(repeat_filename1,S)),fill = TRUE)

In [108]:
repeat_result_dt[,unique(Name)]

[1] "AMK258"   "CRWT145"  "BW96"     "CRWT216"  "BW47"     "AMK275"  
 [7] "AMK204"   "AMK156"   "SCN146"   "AMK196"   "AMK198"   "BW239"   
[13] "SCN13"    "BW214"    "SCN71"    "BW277"    "CRWT232"  "SCN310"  
[19] "AMK20"    "CRWT345"  "OT21"     "BS73"     "CRWT300"  "CRWT307" 
[25] "BW92"     "BW279"    "BW33"     "OT36"     "BW219"    "CRWT271" 
[31] "AMK293"   "CRWT293"  "SCN136"   "AMK212"   "BW286"    "OT220"   
[37] "CRWT196"  "AMK325"   "BW218"    "AMK126"   "CRWT42"   "CRWT318" 
[43] "AMK180"   "OT37"     "AMK185"   "BW217"    "BW192"    "AMK320"  
[49] "BS47"     "CRWT310"  "CRWT151"  "AMK330"   "BW237"    "OT8"     
[55] "BW264"    "AMK176"   "BW287"    "CRWT240"  "AMK207"   "SCN290"  
[61] "CRWT1255" "AMK76"    "BW265"    "CRWT316"  "AMK1154"  "AMK184"  
[67] "CRWT65"   "SCN265"   "CRWT1244" "AMK114"   "AMK157"   "AMK69"   
[73] "SCN131"   "CRWT274"  "BW244"    "CRWT336"  "AMK188"   "OT1026"  
[79] "BW256"    "SCN142"   "AMK1156"  "AMK22"    "CRWT1258" "BW155"   
[85] "OT30"     "AMK214"   "AMK199"   "SCN1115"  "CRWT38"   "IRC"     
[91] "N T"      "NT"

In [109]:
summary(repeat_result_dt)

  qPCR_well             Name                 Cp            gene          
 Length:1470        Length:1470        Min.   :17.78   Length:1470       
 Class :character   Class :character   1st Qu.:23.21   Class :character  
 Mode  :character   Mode  :character   Median :25.23   Mode  :character  
                                       Mean   :26.27                     
                                       3rd Qu.:29.73                     
                                       Max.   :35.80                     
                                       NA's   :594                       
    plate           sample_well       
 Length:1470        Length:1470       
 Class :character   Class :character  
 Mode  :character   Mode  :character  
                                      
                                      
                                      
                                      

### Read Workbook2

In [120]:
repeat_filename2

[1] "GEP_final_repeat_qPCR_results_omit_GEP3+4.xlsx"

In [121]:
(sheetnames <- getSheetNames(repeat_filename2))

[1] "Notes"             "RNA plate order"   "96well to 384well"
[4] "GEPfinal_VRN2"     "GEP_final_egf1a"   "GEPfinal_ta54227"

In [122]:
repeat2_result_dt <- rbindlist(lapply(sheetnames[4:6],function(S) read_qPCR(repeat_filename2,S)),fill = TRUE)

In [124]:
head(repeat_result_dt)
head(repeat2_result_dt)

,qPCR_well,Name,Cp,gene,plate,sample_well
1,A1,AMK258,NA,VRN1,GEP,A1
2,A2,AMK258,NA,VRN1,GEP,A1
3,B1,AMK258,NA,VRN1,GEP,A1
4,A19,CRWT145,NA,VRN1,GEP,A10
5,A20,CRWT145,NA,VRN1,GEP,A10
6,B19,CRWT145,NA,VRN1,GEP,A10


,qPCR_well,Name,Cp,gene,plate,sample_well,Include,Ta.Cp
1,C1,BS164,31.27,NA,GEPfinal,B1,NA,NA
2,C2,BS164,NA,NA,GEPfinal,B1,NA,NA
3,D1,BS164,31.3,NA,GEPfinal,B1,NA,NA
4,C5,CRWT333,33.59,NA,GEPfinal,B3,NA,NA
5,C6,CRWT333,32.76,NA,GEPfinal,B3,NA,NA
6,D5,CRWT333,NA,NA,GEPfinal,B3,NA,NA


To Rbind with Set 1 need names to match

```
'Plate' 'sample_well' 'Name' 'EGF1a' 'Ta54227' 'VRN1' 'VRN2' 'VRN3' 'VRN1_TA' 'VRN2_TA' 'VRN3_TA' 'Col' 'Row' 'Experiment' 'Plot.ID.Old' 'Plot.IDNew' 'SampleDate' 'Vern' 'Cult' 'Pp' 'TaNorm'
```

### Calculate Means

First check the top

In [150]:
repeat_result_dt %>%
    arrange(Name) %>% 
    spread(gene,Cp) %>% 
    group_by(Name,sample_well) %>% head

,Name,sample_well,qPCR_well,plate,ex_sample_plate,ex_sample_plate_well,EGF1a,Ta54227,VRN1,VRN2,VRN3
1,AMK114,G11,M21,GEP,GEP5,A7,19.76,22.42,22.51,NA,24.56
2,AMK114,G11,M22,GEP,GEP5,A7,NA,22.36,22.48,NA,24.7
3,AMK114,G11,N21,GEP,GEP5,A7,19.71,22.48,22.43,NA,24.64
4,AMK1154,F7,K13,GEP,GEP7,8C,22.98,25.44,26.61,31.9,NA
5,AMK1154,F7,K14,GEP,GEP7,8C,23.09,25.39,26.51,NA,NA
6,AMK1154,F7,L13,GEP,GEP7,8C,NA,25.24,26.64,33.47,NA


In [147]:
repeat_result_dt %>%
    arrange(Name) %>% 
    spread(gene,Cp) %>% 
    group_by(Name,sample_well) %>%
     summarise_each(funs(mean(.,na.rm=TRUE)),7:11) %>% head
    

,Name,sample_well,EGF1a,Ta54227,VRN1,VRN2,VRN3
1,AMK114,G11,19.735,22.42,22.47333,NaN,24.63333
2,AMK1154,F7,23.035,25.35667,26.58667,32.685,NaN
3,AMK1156,H10,23.35667,27.61,30.375,33.87,NaN
4,AMK126,D2,22.75333,25.225,26.97,33.125,31.41667
5,AMK156,A5,21.79667,24.185,NaN,30.61,NaN
6,AMK157,G12,24.19,26.77,34.465,32.25333,NaN


Next need to get deltadelta Ct and IRC corrections and rbind onto Set1

In [148]:
repeat_result_dt %>%
    arrange(Name) %>% 
    spread(gene,Cp) %>% 
    group_by(Name,sample_well) %>%
     summarise_each(funs(mean(.,na.rm=TRUE)),7:11) %>%
    mutate(VRN1_TA=2^-(VRN1-Ta54227)) %>%
    mutate(VRN2_TA=2^-(VRN2-Ta54227)) %>%
    mutate(VRN3_TA=2^-(VRN3-Ta54227)) %>% head

,Name,sample_well,EGF1a,Ta54227,VRN1,VRN2,VRN3,VRN1_TA,VRN2_TA,VRN3_TA
1,AMK114,G11,19.735,22.42,22.47333,NaN,24.63333,0.9637071,NaN,0.2156355
2,AMK1154,F7,23.035,25.35667,26.58667,32.685,NaN,0.4263174,0.006222313,NaN
3,AMK1156,H10,23.35667,27.61,30.375,33.87,NaN,0.1471133,0.01304825,NaN
4,AMK126,D2,22.75333,25.225,26.97,33.125,31.41667,0.2983339,0.004186615,0.01368115
5,AMK156,A5,21.79667,24.185,NaN,30.61,NaN,NaN,0.01163811,NaN
6,AMK157,G12,24.19,26.77,34.465,32.25333,NaN,0.004825854,0.02235384,NaN


In [155]:
repeat_mean_dt <- repeat_result_dt %>%
    arrange(Name) %>% 
    spread(gene,Cp) %>% 
    group_by(Name,sample_well) %>%
     summarise_each(funs(mean(.,na.rm=TRUE)),7:11) %>%
    mutate(VRN1_TA=2^-(VRN1-Ta54227)) %>%
    mutate(VRN2_TA=2^-(VRN2-Ta54227)) %>%
    mutate(VRN3_TA=2^-(VRN3-Ta54227))
head(repeat_mean_dt)

,Name,sample_well,EGF1a,Ta54227,VRN1,VRN2,VRN3,VRN1_TA,VRN2_TA,VRN3_TA
1,AMK114,G11,19.735,22.42,22.47333,NaN,24.63333,0.9637071,NaN,0.2156355
2,AMK1154,F7,23.035,25.35667,26.58667,32.685,NaN,0.4263174,0.006222313,NaN
3,AMK1156,H10,23.35667,27.61,30.375,33.87,NaN,0.1471133,0.01304825,NaN
4,AMK126,D2,22.75333,25.225,26.97,33.125,31.41667,0.2983339,0.004186615,0.01368115
5,AMK156,A5,21.79667,24.185,NaN,30.61,NaN,NaN,0.01163811,NaN
6,AMK157,G12,24.19,26.77,34.465,32.25333,NaN,0.004825854,0.02235384,NaN


### Convert NaN to 0

In [157]:
Nanto0 <- function(X) ifelse(is.nan(X),0,X)
repeat_mean_dt %>%
    mutate_each(.,funs(Nanto0),matches("_TA")) %>%
    head

,Name,sample_well,EGF1a,Ta54227,VRN1,VRN2,VRN3,VRN1_TA,VRN2_TA,VRN3_TA
1,AMK114,G11,19.735,22.42,22.47333,NaN,24.63333,0.9637071,0,0.2156355
2,AMK1154,F7,23.035,25.35667,26.58667,32.685,NaN,0.4263174,0.006222313,0
3,AMK1156,H10,23.35667,27.61,30.375,33.87,NaN,0.1471133,0.01304825,0
4,AMK126,D2,22.75333,25.225,26.97,33.125,31.41667,0.2983339,0.004186615,0.01368115
5,AMK156,A5,21.79667,24.185,NaN,30.61,NaN,0,0.01163811,0
6,AMK157,G12,24.19,26.77,34.465,32.25333,NaN,0.004825854,0.02235384,0


In [158]:
repeat_mean_dt %<>%
    mutate_each(.,funs(Nanto0),matches("_TA")) 

### Join Repeats DT onto cDNA DT

> 'sample_well' joins to 'repeat_plate_template_well'

In [159]:
merge(repeat_mean_dt,cDNA_dt,by.x=c('Name','sample_well'),by.y=c('Name','repeat_plate_template_well')) %>% head

,Name,sample_well,EGF1a,Ta54227,VRN1,VRN2,VRN3,VRN1_TA,VRN2_TA,VRN3_TA,ex_sample_plate,ex_sample_plate_well
1,AMK114,G11,19.735,22.42,22.47333,NaN,24.63333,0.9637071,0,0.2156355,GEP5,A7
2,AMK1154,F7,23.035,25.35667,26.58667,32.685,NaN,0.4263174,0.006222313,0,GEP7,8C
3,AMK1156,H10,23.35667,27.61,30.375,33.87,NaN,0.1471133,0.01304825,0,GEP9,A9
4,AMK126,D2,22.75333,25.225,26.97,33.125,31.41667,0.2983339,0.004186615,0.01368115,GEP2,5D
5,AMK156,A5,21.79667,24.185,NaN,30.61,NaN,0,0.01163811,0,GEP7,1E
6,AMK157,G12,24.19,26.77,34.465,32.25333,NaN,0.004825854,0.02235384,0,GEP8,D10


In [164]:
repeat_mean_dt <- merge(repeat_mean_dt,cDNA_dt,by.x=c('Name','sample_well'),by.y=c('Name','repeat_plate_template_well'))

### Read the sample sheet

In [161]:
sample_sheetdt <- fread('./sample_sheet.csv')
sample_sheetdt$SampleDate <- as.Date(sample_sheetdt$SampleDate)

Need to join
> ex_sample_plate + ex_sample_plate_well to Plate + sample_well

but need to fix row/column order first

In [176]:
repeat_mean_dt$ex_sample_plate_well <-  gsub("^(\\d+)(\\w)","\\2\\1",repeat_mean_dt$ex_sample_plate_well)

In [177]:
head(repeat_mean_dt)
head(sample_sheetdt)

,Name,sample_well,EGF1a,Ta54227,VRN1,VRN2,VRN3,VRN1_TA,VRN2_TA,VRN3_TA,ex_sample_plate,ex_sample_plate_well
1,AMK114,G11,19.735,22.42,22.47333,NaN,24.63333,0.9637071,0,0.2156355,GEP5,A7
2,AMK1154,F7,23.035,25.35667,26.58667,32.685,NaN,0.4263174,0.006222313,0,GEP7,C8
3,AMK1156,H10,23.35667,27.61,30.375,33.87,NaN,0.1471133,0.01304825,0,GEP9,A9
4,AMK126,D2,22.75333,25.225,26.97,33.125,31.41667,0.2983339,0.004186615,0.01368115,GEP2,D5
5,AMK156,A5,21.79667,24.185,NaN,30.61,NaN,0,0.01163811,0,GEP7,E1
6,AMK157,G12,24.19,26.77,34.465,32.25333,NaN,0.004825854,0.02235384,0,GEP8,D10


,Plate,Col,Row,Experiment,Plot.ID.Old,Plot.IDNew,SampleDate,Vern,Cult,Pp,sample_well
1,GEP1,1,A,3,105,300105,2014-02-04,None,Amarok,16,A1
2,GEP1,10,A,3,2,300002,2014-03-04,None,Saracen,16,A10
3,GEP1,11,A,3,190,300190,2014-03-04,None,Amarok,16,A11
4,GEP1,12,A,3,123,300123,2014-03-11,None,Saracen,16,A12
5,GEP1,2,A,3,262,300262,2014-02-04,None,CRWT153,16,A2
6,GEP1,3,A,3,326,300326,2014-02-11,None,Saracen,16,A3


In [180]:
merge(repeat_mean_dt,sample_sheetdt,by.x=c('ex_sample_plate','ex_sample_plate_well'),by.y=c('Plate','sample_well')) %>% head

,ex_sample_plate,ex_sample_plate_well,Name,sample_well,EGF1a,Ta54227,VRN1,VRN2,VRN3,VRN1_TA,VRN2_TA,VRN3_TA,Col,Row,Experiment,Plot.ID.Old,Plot.IDNew,SampleDate,Vern,Cult,Pp
1,GEP1,A6,BW96,A11,21.07333,24.51,NaN,28.45667,NaN,0,0.06485373,0,6,A,3,96,300096,2014-02-18,None,Batten Win,16
2,GEP1,F6,OT220,C9,23.39667,25.945,28.78,32.475,NaN,0.1401458,0.01082117,0,6,F,3,220,300220,2014-02-18,None,Otane,16
3,GEP1,G12,AMK258,A1,21.97,24.85333,NaN,31.015,NaN,0,0.01396862,0,12,G,3,258,300258,2014-03-11,None,Amarok,16
4,GEP2,A3,AMK320,E1,22.62,25.78,NaN,32.44333,NaN,0,0.009865902,0,3,A,3,320,300320,2014-03-18,None,Amarok,16
5,GEP2,A5,BW219,C2,22.94667,25.57667,28.7,32.2,32.835,0.114758,0.01014327,0.006531666,5,A,3,219,300219,2014-03-25,None,Batten Win,16
6,GEP2,A6,SCN71,B11,21.48,25.82333,30.44,31.305,NaN,0.040761,0.02237968,0,6,A,3,71,300071,2014-04-01,None,Saracen,16


In [181]:
merge(repeat_mean_dt,sample_sheetdt,
      by.x=c('ex_sample_plate','ex_sample_plate_well'),
      by.y=c('Plate','sample_well')) %>%
        write_csv('./qPCRMean_Repeat1.csv')